trainval / test


jumping, other, phoning, playinginstrument, reading, ridingbike, ridinghorse, running, takingphoto, usingcomputer, walking

In [ ]:
import os 
import sys
import cv2
import caffe
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import sklearn.svm
import sklearn.linear_model
import sklearn.kernel_approximation # RBFSampler

In [ ]:
import caffe
import tensorflow as tf

In [ ]:
sys.path.append(os.path.join(os.getcwd(), 'datasets/pascal/'))
import pascal

In [ ]:
RESNET_PATH = '/home/omer/MyRoot/work/grideas/pretrained/resnet152'

In [ ]:
images, classnames, poses = pascal.annotations()

In [ ]:
caffe.set_mode_cpu()
resnet = caffe.Net(os.path.join(RESNET_PATH, 'ResNet-152-deploy.prototxt'), caffe.TEST)
resnet.copy_from(os.path.join(RESNET_PATH, 'ResNet-152-model.caffemodel'))

In [ ]:
img_name = images.keys()[np.random.randint(len(images.keys()))]
img = cv2.cvtColor(cv2.imread(pascal.imagefile(img_name), cv2.IMREAD_COLOR), cv2.COLOR_RGB2BGR)

In [ ]:
offset = lambda dim, size: int(np.floor((dim-size)*0.5))
sizeX, sizeY = 224, 224
offsetY, offsetX = offset(img.shape[0], sizeY), offset(img.shape[1], sizeX)
cropped = img[offsetY:offsetY+sizeY, offsetX:offsetX+sizeX, :].squeeze()

In [ ]:
plt.imshow(cropped)
plt.show()

In [ ]:
im_input = np.zeros((1, 3, 224, 224))
cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
for c in xrange(cropped.shape[2]):
    im_input[0, c :, :] = cropped[:, :, c]
resnet.blobs['data'].data[...] = im_input

In [ ]:
# result = resent.forward(end='res5a')
result = resnet.forward(start='conv1')
# /esult = resnet.forward()

In [ ]:
print np.argmax(result['prob'])

# And... TensorFlow

In [ ]:
def resnet_load(resnet_path=RESNET_PATH):
    caffe.set_mode_cpu()
    resnet = caffe.Net(os.path.join(resnet_path, 'ResNet-152-deploy.prototxt'), caffe.TEST)
    resnet.copy_from(os.path.join(resnet_path, 'ResNet-152-model.caffemodel'))    
    return resnet

def resnet_crop(img):
    offset = lambda dim, size: int(np.floor((dim-size)*0.5))
    sizeX, sizeY = 224, 224
    offsetY, offsetX = offset(img.shape[0], sizeY), offset(img.shape[1], sizeX)
    cropped = img[offsetY:offsetY+sizeY, offsetX:offsetX+sizeX, :].squeeze()
    return cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)

def resnet_setinputs(resnet_model, data):
    im_input = np.zeros((1, 3, 224, 224))
    for c in xrange(data.shape[2]):
        im_input[0, c :, :] = data[:, :, c]
    resnet_model.blobs['data'].data[...] = im_input
    return resnet_model

def get_image(images, image_name=None):
    if image_name is None:
        image_name = images.keys()[np.random.randint(len(images.keys()))]
    return cv2.cvtColor(cv2.imread(pascal.imagefile(image_name), cv2.IMREAD_COLOR), cv2.COLOR_RGB2BGR)

def get_features(image, caffe_model, blob_name):
    resnet_model = resnet_setinputs(caffe_model, resnet_crop(image))
    return resnet.forward(end=blob_name)[blob_name][0, :, :, :]

In [ ]:
counter = 0
Xlst = []
Ylst = []

for imgname, info in images.items():
    Xlst.append(imgname)
    Ylst.append('person' in [curr_obj['classname'] for curr_obj in info['objects']])
    counter += 1
    if counter > 1000:
        break

In [ ]:
svm = sklearn.linear_model.SGDClassifier(loss='hinge')

BATCH_SIZE = 10
for idx in xrange(BATCH_SIZE):
    print idx, Xlst[idx]
    features = get_features(get_image(images, Xlst[idx]), resnet, 'res5a')
    print features.shape
    break

In [ ]:
for nm in Xlst:
    get_image(images, nm)

In [ ]:
resnet = resnet_load(RESNET_PATH)

In [ ]:
features = get_features(get_image(images), resnet, 'res5a')

In [ ]:
features.flatten().shape

In [ ]:
# svm = sklearn.svm.SVC(C=1.0, kernel='rbf')
# SGDClassifier

In [ ]:
sess = tf.InteractiveSession()